In [11]:
import pandas as pd
import audioread
import numpy as np
import librosa
import math
import re
import os
from multiprocessing import Pool
import warnings
warnings.filterwarnings('ignore')

In [25]:
data_audio = pd.read_csv('../Handout/train_last.csv')

In [26]:
data_audio

,Label,File
0,5,PAEP-004667_noise.wav
1,3,PAEP-002135.wav
2,3,PAEP-003204.wav
3,5,PAEP-001005_shift.wav
4,0,PAEP-004512_shift.wav
...,...,...
15682,2,PAEP-004048_noise.wav
15683,4,PAEP-002564_shift.wav
15684,3,PAEP-001220.wav
15685,2,PAEP-003784.wav


In [27]:
def png_to_wav(fn):
    return fn.replace(".png",".wav")
data_audio["File"]=data_audio["File"].apply(png_to_wav)

In [28]:
#Mã label: neutral(0), happy(1), sad(2), anger(3), fear(4), disgust(5)
label = {0:'neutral', 1:'happy', 2:'sad', 3:'anger', 4:'fear', 5:'disgust'}
data_audio.head()

,Label,File
0,5,PAEP-004667_noise.wav
1,3,PAEP-002135.wav
2,3,PAEP-003204.wav
3,5,PAEP-001005_shift.wav
4,0,PAEP-004512_shift.wav


In [29]:
def getfeature(fname):
    timeseries_length=128
    hop_length = 512
    data = np.zeros((timeseries_length, 33), dtype=np.float64)

    y, sr = librosa.load(fname)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)
    spectral_center = librosa.feature.spectral_centroid(y=y, sr=sr, hop_length=hop_length)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=hop_length)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr, hop_length=hop_length)

    filelength = timeseries_length if mfcc.shape[1] >= timeseries_length else mfcc.shape[1]
    

    data[-filelength:, 0:13] = mfcc.T[0:timeseries_length, :]
    data[-filelength:, 13:14] = spectral_center.T[0:timeseries_length, :]
    data[-filelength:, 14:26] = chroma.T[0:timeseries_length, :]
    data[-filelength:, 26:33] = spectral_contrast.T[0:timeseries_length, :]

    return data

In [30]:
def processtrain(fname):
    data = getfeature(fname)
    name = fname.split('/')[-1].split('.')[0] + '.wav'
    emotion = data_audio[data_audio["File"] == name]["Label"].item()
    return data, emotion

def processtest(fname):
    data = getfeature(fname)
    name = fname.split('/')[-1]   
    return data, name

In [31]:
def absoluteFilePaths(directory):
    for dirpath,_,filenames in os.walk(directory):
        for f in filenames:
            yield os.path.abspath(os.path.join(dirpath, f))

In [32]:
def train():
    files = list(absoluteFilePaths('../Handout/Train1/'))
    p = Pool(4)    
    data = p.map(processtrain, files)
    X = [data[i][0] for i in range(len(data))]
    X = np.asarray(X)

    emotion = [data[i][1] for i in range(len(data))]

    np.savez('../npz/train_last', X=X, emotion=emotion)
    print('Save train npz successfully !')
    
def test():
    files = list(absoluteFilePaths('../Public_Test/Public_Test/'))
    p = Pool(4) 
    data = p.map(processtest, files)
    
    X = [data[i][0] for i in range(len(data))]
    X = np.asarray(X)
    
    name = [data[i][1] for i in range(len(data))]
    np.savez('../npz/test_last', X=X, name=name)
    print('Save test npz successfully !')

In [33]:
train()
test()

PAEP-005219.wav
PAEP-002889_shift.wav
PAEP-000298_noise.wav
PAEP-004667_noise.wav
PAEP-001420_noise.wav
PAEP-002135.wav
PAEP-001013_noise.wav
PAEP-004962_noise.wav
PAEP-001687_shift.wav
PAEP-002653_noise.wav
PAEP-003204.wav
PAEP-004180.wav
PAEP-004619_noise.wav
PAEP-002425.wav
PAEP-001005_shift.wav
PAEP-002209_noise.wav
PAEP-002601.wav
PAEP-004512_shift.wav
PAEP-001052.wav
PAEP-004713_shift.wav
PAEP-002398_shift.wav
PAEP-002380_noise.wav
PAEP-000551_noise.wav
PAEP-003717.wav
PAEP-001177.wav
PAEP-004256_shift.wav
PAEP-001457.wav
PAEP-003653_shift.wav
PAEP-001390.wav
PAEP-001368.wav
PAEP-002259.wav
PAEP-001530.wav
PAEP-001815_shift.wav
PAEP-002253.wav
PAEP-003633.wav
PAEP-001102_noise.wav
PAEP-001953_shift.wav
PAEP-001922_noise.wav
PAEP-003899_shift.wav
PAEP-004851.wav
PAEP-000764_noise.wav
PAEP-001372.wav
PAEP-000008_shift.wav
PAEP-001780_shift.wav
PAEP-004376_noise.wav
PAEP-004129_noise.wav
PAEP-002963_shift.wav
PAEP-001277.wav
PAEP-000105.wav
PAEP-002514_shift.wav
PAEP-003055.wav
PAEP

PAEP-001190_noise.wav
PAEP-000438_noise.wav
PAEP-000942.wav
PAEP-004015.wav
PAEP-004380_noise.wav
PAEP-002828.wav
PAEP-004706_noise.wav
PAEP-001658.wav
PAEP-003874_noise.wav
PAEP-001897.wav
PAEP-003092_noise.wav
PAEP-000374_noise.wav
PAEP-003743_noise.wav
PAEP-004941_noise.wav
PAEP-003300_noise.wav
PAEP-001826.wav
PAEP-000858_noise.wav
PAEP-000311.wav
PAEP-000046_noise.wav
PAEP-003041_shift.wav
PAEP-001306.wav
PAEP-000363_shift.wav
PAEP-000569_shift.wav
PAEP-002281_noise.wav
PAEP-004455_noise.wav
PAEP-004627_shift.wav
PAEP-003438_shift.wav
PAEP-000180_shift.wav
PAEP-000173_noise.wav
PAEP-000920_shift.wav
PAEP-001417_noise.wav
PAEP-002949_shift.wav
PAEP-000964.wav
PAEP-000717.wav
PAEP-001462_shift.wav
PAEP-003102.wav
PAEP-001434_noise.wav
PAEP-003719_noise.wav
PAEP-004352.wav
PAEP-004124.wav
PAEP-001138_noise.wav
PAEP-004695.wav
PAEP-003533_shift.wav
PAEP-002642.wav
PAEP-004942_shift.wav
PAEP-001511_noise.wav
PAEP-001419_shift.wav
PAEP-004656_shift.wav
PAEP-004385.wav
PAEP-002587_noise.

PAEP-004251.wav
PAEP-003800.wav
PAEP-004228.wav
PAEP-000687.wav
PAEP-003964.wav
PAEP-000133_noise.wav
PAEP-000435_shift.wav
PAEP-005074_shift.wav
PAEP-001444_noise.wav
PAEP-001263_shift.wav
PAEP-004433_shift.wav
PAEP-000802_shift.wav
PAEP-003173_shift.wav
PAEP-001693_shift.wav
PAEP-002170.wav
PAEP-001643.wav
PAEP-001689_shift.wav
PAEP-002678_noise.wav
PAEP-003659.wav
PAEP-000945_noise.wav
PAEP-002204_noise.wav
PAEP-003618_shift.wav
PAEP-005149.wav
PAEP-004536.wav
PAEP-001102_shift.wav
PAEP-003197_shift.wav
PAEP-004026.wav
PAEP-005025_shift.wav
PAEP-002389.wav
PAEP-000010.wav
PAEP-001712_shift.wav
PAEP-001328_shift.wav
PAEP-001475_shift.wav
PAEP-004078_shift.wav
PAEP-001588_noise.wav
PAEP-004173_noise.wav
PAEP-000088_noise.wav
PAEP-004951.wav
PAEP-000441_shift.wav
PAEP-001773.wav
PAEP-003579_shift.wav
PAEP-003029_shift.wav
PAEP-002318_noise.wav
PAEP-002416_shift.wav
PAEP-004807.wav
PAEP-002369.wav
PAEP-001930_shift.wav
PAEP-001556.wav
PAEP-004947.wav
PAEP-002047_noise.wav
PAEP-001748.wa

PAEP-001571.wav
PAEP-003562_noise.wav
PAEP-002397_noise.wav
PAEP-000184_noise.wav
PAEP-000743.wav
PAEP-003612.wav
PAEP-002939_noise.wav
PAEP-000544_noise.wav
PAEP-000658_shift.wav
PAEP-000922_shift.wav
PAEP-004539_shift.wav
PAEP-001116.wav
PAEP-004111_shift.wav
PAEP-000923_noise.wav
PAEP-003992_noise.wav
PAEP-000634_shift.wav
PAEP-005087.wav
PAEP-001721.wav
PAEP-000129_shift.wav
PAEP-000034_noise.wav
PAEP-003375_noise.wav
PAEP-000477_noise.wav
PAEP-005163_noise.wav
PAEP-003627_shift.wav
PAEP-002563_shift.wav
PAEP-004058_shift.wav
PAEP-001234_noise.wav
PAEP-004987_shift.wav
PAEP-000047_shift.wav
PAEP-002729.wav
PAEP-001676_noise.wav
PAEP-004586.wav
PAEP-002938_noise.wav
PAEP-004602_noise.wav
PAEP-001625_noise.wav
PAEP-000532_noise.wav
PAEP-002644.wav
PAEP-003539_shift.wav
PAEP-004520_shift.wav
PAEP-001149_shift.wav
PAEP-002010_shift.wav
PAEP-000531_shift.wav
PAEP-001071_shift.wav
PAEP-001694.wav
PAEP-000524_shift.wav
PAEP-004256_noise.wav
PAEP-001720.wav
PAEP-003010_shift.wav
PAEP-00366

PAEP-001796_shift.wav
PAEP-002809.wav
PAEP-003924_noise.wav
PAEP-001936.wav
PAEP-005157_noise.wav
PAEP-003022_shift.wav
PAEP-005065_shift.wav
PAEP-004367_shift.wav
PAEP-004756_noise.wav
PAEP-001211_shift.wav
PAEP-002661.wav
PAEP-002768_noise.wav
PAEP-002351.wav
PAEP-001978_noise.wav
PAEP-003528_noise.wav
PAEP-005057_noise.wav
PAEP-002133_noise.wav
PAEP-000026.wav
PAEP-000013.wav
PAEP-000158_noise.wav
PAEP-001484.wav
PAEP-003867_shift.wav
PAEP-003358_noise.wav
PAEP-001396_shift.wav
PAEP-000268_noise.wav
PAEP-004754_noise.wav
PAEP-003671_noise.wav
PAEP-001271_noise.wav
PAEP-002693_noise.wav
PAEP-001082_noise.wav
PAEP-003555_shift.wav
PAEP-004217.wav
PAEP-000948.wav
PAEP-002454_noise.wav
PAEP-000345_noise.wav
PAEP-002917.wav
PAEP-005138_shift.wav
PAEP-001220_shift.wav
PAEP-001439_noise.wav
PAEP-001665_shift.wav
PAEP-004085.wav
PAEP-001254_noise.wav
PAEP-000515_noise.wav
PAEP-001206_noise.wav
PAEP-002341.wav
PAEP-002758.wav
PAEP-003460_shift.wav
PAEP-001178.wav
PAEP-001556_noise.wav
PAEP-0

...

PAEP-003564.wav
PAEP-000381_noise.wav
PAEP-002790_noise.wav
PAEP-004434_shift.wav
PAEP-001862_noise.wav
PAEP-003152_shift.wav
PAEP-004494_shift.wav
PAEP-005031_noise.wav
PAEP-003568.wav
PAEP-000320_shift.wav
PAEP-003248_shift.wav
PAEP-001286.wav
PAEP-000053.wav
PAEP-000689_noise.wav
PAEP-002517_shift.wav
PAEP-002616.wav
PAEP-001057_noise.wav
PAEP-005171_shift.wav
PAEP-003073_shift.wav
PAEP-000273.wav
PAEP-003394_shift.wav
PAEP-001093_shift.wav
PAEP-003026_noise.wav
PAEP-004843_shift.wav
PAEP-001675_shift.wav
PAEP-004588_shift.wav
PAEP-004423_shift.wav
PAEP-004091.wav
PAEP-001326_noise.wav
PAEP-000521.wav
PAEP-003358.wav
PAEP-001044.wav
PAEP-004887.wav
PAEP-004956_noise.wav
PAEP-002810_noise.wav
PAEP-000143_shift.wav
PAEP-002155_noise.wav
PAEP-001424.wav
PAEP-000197_noise.wav
PAEP-000209.wav
PAEP-004715_noise.wav
PAEP-004301_shift.wav
PAEP-002244_noise.wav
PAEP-004609.wav
PAEP-003538_noise.wav
PAEP-002715.wav
PAEP-000641.wav
PAEP-003853_noise.wav
PAEP-000138.wav
PAEP-004179_noise.wav
PA

PAEP-003670_noise.wav
PAEP-003765_shift.wav
PAEP-004234_shift.wav
PAEP-002632_noise.wav
PAEP-000730_shift.wav
PAEP-001656_noise.wav
PAEP-005108_shift.wav
PAEP-002541.wav
PAEP-003784_noise.wav
PAEP-000500_shift.wav
PAEP-002867_shift.wav
PAEP-003663_noise.wav
PAEP-004055_noise.wav
PAEP-000491.wav
PAEP-002413_noise.wav
PAEP-000361.wav
PAEP-002424.wav
PAEP-001419_noise.wav
PAEP-001885.wav
PAEP-001047_shift.wav
PAEP-000136_noise.wav
PAEP-001242.wav
PAEP-004453_shift.wav
PAEP-002614_noise.wav
PAEP-001036.wav
PAEP-000510_shift.wav
PAEP-001539_shift.wav
PAEP-000846_shift.wav
PAEP-004692_noise.wav
PAEP-000911_noise.wav
PAEP-004459_shift.wav
PAEP-004340_noise.wav
PAEP-002785_noise.wav
PAEP-004166_noise.wav
PAEP-001737_noise.wav
PAEP-000803_noise.wav
PAEP-000119_noise.wav
PAEP-003599.wav
PAEP-002514.wav
PAEP-003137_noise.wav
PAEP-000780_noise.wav
PAEP-002176_noise.wav
PAEP-001899.wav
PAEP-002520.wav
PAEP-004661.wav
PAEP-004995.wav
PAEP-004223.wav
PAEP-001945_noise.wav
PAEP-001280.wav
PAEP-005043_

PAEP-002578.wav
PAEP-003446.wav
PAEP-003755_noise.wav
PAEP-003755_shift.wav
PAEP-004079.wav
PAEP-000648.wav
PAEP-002817.wav
PAEP-004782.wav
PAEP-001766_noise.wav
PAEP-003050_shift.wav
PAEP-000526_noise.wav
PAEP-001618_noise.wav
PAEP-003343_noise.wav
PAEP-002640_noise.wav
PAEP-003062_shift.wav
PAEP-000236_shift.wav
PAEP-003608_noise.wav
PAEP-004998.wav
PAEP-000014.wav
PAEP-000380.wav
PAEP-000520_noise.wav
PAEP-000222_noise.wav
PAEP-001011.wav
PAEP-003779_noise.wav
PAEP-000907_shift.wav
PAEP-000990.wav
PAEP-000313_noise.wav
PAEP-003724_shift.wav
PAEP-001963_noise.wav
PAEP-001850_shift.wav
PAEP-000208.wav
PAEP-003643_shift.wav
PAEP-002204_shift.wav
PAEP-001740_noise.wav
PAEP-001927.wav
PAEP-002433_shift.wav
PAEP-002213_noise.wav
PAEP-005155.wav
PAEP-004918_noise.wav
PAEP-002052.wav
PAEP-000112_noise.wav
PAEP-002461_shift.wav
PAEP-001445_noise.wav
PAEP-001844_shift.wav
PAEP-001281_noise.wav
PAEP-002087.wav
PAEP-001682.wav
PAEP-002208_shift.wav
PAEP-004172_shift.wav
PAEP-000109.wav
PAEP-001

PAEP-004551_noise.wav
PAEP-003369_noise.wav
PAEP-001836_noise.wav
PAEP-002332_shift.wav
PAEP-002654.wav
PAEP-000656.wav
PAEP-000557.wav
PAEP-004344_noise.wav
PAEP-001059_shift.wav
PAEP-003623_shift.wav
PAEP-002185.wav
PAEP-003038.wav
PAEP-003451_shift.wav
PAEP-003947.wav
PAEP-004296_shift.wav
PAEP-002255_shift.wav
PAEP-000397_shift.wav
PAEP-003162_noise.wav
PAEP-002259_noise.wav
PAEP-001171.wav
PAEP-004126_shift.wav
PAEP-002621_noise.wav
PAEP-000217_shift.wav
PAEP-002495.wav
PAEP-003610_noise.wav
PAEP-003634.wav
PAEP-004866_noise.wav
PAEP-002732_noise.wav
PAEP-001054.wav
PAEP-003868_noise.wav
PAEP-003164_noise.wav
PAEP-003118_noise.wav
PAEP-004502_shift.wav
PAEP-005044_shift.wav
PAEP-003660.wav
PAEP-005076_noise.wav
PAEP-003654_noise.wav
PAEP-000484.wav
PAEP-004860_shift.wav
PAEP-002369_noise.wav
PAEP-000966_shift.wav
PAEP-003703_noise.wav
PAEP-002112.wav
PAEP-003546_shift.wav
PAEP-000717_noise.wav
PAEP-001966_noise.wav
PAEP-002269_noise.wav
PAEP-000213_noise.wav
PAEP-002477.wav
PAEP-0

PAEP-001349_noise.wav
PAEP-001255_noise.wav
PAEP-003141_shift.wav
PAEP-002782_shift.wav
PAEP-001063_noise.wav
PAEP-001261_shift.wav
PAEP-000249_noise.wav
PAEP-000698_shift.wav
PAEP-003269_noise.wav
PAEP-004081_shift.wav
PAEP-003645_noise.wav
PAEP-004315.wav
PAEP-003711_shift.wav
PAEP-002233.wav
PAEP-002325_shift.wav
PAEP-001825.wav
PAEP-002930.wav
PAEP-003220_noise.wav
PAEP-005191.wav
PAEP-004711.wav
PAEP-001161_noise.wav
PAEP-003425_noise.wav
PAEP-001024_noise.wav
PAEP-001337_shift.wav
PAEP-000603_shift.wav
PAEP-001844_noise.wav
PAEP-003261_shift.wav
PAEP-001525_noise.wav
PAEP-004336_shift.wav
PAEP-004074_noise.wav
PAEP-001802_noise.wav
PAEP-001543.wav
PAEP-000138_noise.wav
PAEP-000071_noise.wav
PAEP-003353.wav
PAEP-002360_shift.wav
PAEP-003915_noise.wav
PAEP-001371.wav
PAEP-005045.wav
PAEP-003132.wav
PAEP-004240.wav
PAEP-002255.wav
PAEP-004845_noise.wav
PAEP-001618_shift.wav
PAEP-001354.wav
PAEP-003485_shift.wav
PAEP-003816.wav
PAEP-003589_shift.wav
PAEP-004154_noise.wav
PAEP-002615_